In [4]:
from bs4 import BeautifulSoup
from pprint import pprint
import requests
from selenium import webdriver
import pymysql
import re
from urllib.parse import urlparse, parse_qs

# mysql db connect
db = pymysql.connect(host='52.78.23.232', user='root', password='readers7', port=3306, db='readers', charset='utf8', cursorclass=pymysql.cursors.DictCursor)
# browser = webdriver.Chrome('C:/Users/KSY/Documents/GitHub/Readers/crawling/chromedriver.exe')
cursor1 = db.cursor()
cursor2 = db.cursor()
cursor3 = db.cursor()
cursor4 = db.cursor()

#웹 페이지를 열고 소스코드를 읽어오는 작업
def readHtml(url):
    # print(url)
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    html.close()
    return soup

### 네이버 웹툰 회차정보 크롤링 ###

sel_url_sql = 'select toon_id, toon_url from toon_info where toon_site = "naver"'
sel_dup_sql = 'select toon_id, epi_name from epi_info where toon_id = %s and epi_name = %s'
ins_epi_info_sql = 'insert into epi_info(toon_id, epi_name, epi_url, epi_thumb_url, epi_date)\
                    values (%s, %s, %s, %s, %s)'

cursor1.execute(sel_url_sql)
result = cursor1.fetchall()
   
for row in result:
    toon_id = row['toon_id']
    toon_url = row['toon_url']
    
    num = 1 # 페이지 탐색용
    have_to_break = 0 # 루프 탈출용
    
    while 1:
        real_url = toon_url+str(num) # 실제 웹툰 링크, 첫 페이지부터
        num = num + 1 # 다음 페이지 탐색을 위해
        soup = readHtml(real_url)

        webtoon_area = soup.find("table", {"class": "viewList"}).findAll('tr')
        
        if (have_to_break == 1):
            break
        
        for index in webtoon_area:
            info = index.find('a')
            if (index.find('a')):
                if (index.find('td', {'class' : 'num'})):
                    info = index.find('a')
                    epi_url = "https://comic.naver.com" + info['href'] # 에피소드 링크 추출
                    img = info.find('img')
                    epi_thumb_url = img['src'] # 썸네일 링크 추출
                    epi_name = img['title'] # 에피소드 제목 추출
                    date = index.find('td', {'class' : 'num'}).get_text()
                    epi_date = date.replace(".", "-") # 업데이트 날짜 추출
                    
                    cursor2.execute(sel_dup_sql, (toon_id, epi_name))
                    if cursor2.fetchall():
                        have_to_break = 1
                        break
    
                    else:
                        cursor3.execute(ins_epi_info_sql, (toon_id, epi_name, epi_url, epi_thumb_url, epi_date))
                        db.commit()
                
            else:
                continue

db.close()